In [3]:
import numpy as np
import re
import math
import random
import matplotlib.pyplot as plt

In [4]:
yelp_input = []
l = []
yelp_path = './yelp_labelled.txt'
with open(yelp_path) as yp:
    line = yp.readline()
    while line:
        yelp_input.append(line.strip())
        line = yp.readline()

In [5]:
def DataSet(ds):
    data = [j[:-3] for j in ds]
    data = [item.lower() for item in data]
    for i in range(len(data)):
        data[i] = re.sub('[^A-Za-z0-9-\s]+', '', data[i])
    k = [j[-1:] for j in ds]
    for i in range(0, len(k)): 
        k[i] = int(k[i])
    ds = list(zip(data , k))
    
    return ds

In [6]:
ds = DataSet(yelp_input)

In [7]:
def dist(ds):
    ds_1 = []
    ds_0 = []
    
    for i in range(len(ds)):
        if ds[i][1] == 1:
            ds_1.append(ds[i])
        else:
            ds_0.append(ds[i])
    
    ds_1 = random.sample(ds_1, len(ds_1))
    
    ds_0 = random.sample(ds_0, len(ds_0))
    
    return ds_1, ds_0

In [8]:
ds_1, ds_0 = dist(ds)

In [9]:
def train_test_split(ds, val):
    index = int(val*len(ds))
    train = ds[:index]
    test = ds[index:]
    
    return train, test

In [10]:
train1, test1 = train_test_split(ds_1, 0.7)
train0, test0 = train_test_split(ds_0, 0.7)

In [11]:
def token_data(input_data):
    words = {}
    tot_words = 0
    for i in range(len(input_data)):
        for w in input_data[i][0].split(" "):
            tot_words += 1
            if(w in words):
                words[w] += 1
            else:
                words[w] = 1
    
    return words, tot_words

In [12]:
word_1, tot_word1 = token_data(train1)
word_0, tot_word0 = token_data(train0)

In [13]:
train = train1 + train0
train_random = random.sample(train, len(train))

test = test1 + test0
test_random = random.sample(test, len(test))

In [14]:
TrainData = []
TrainLabels = []
TestData = []
TestLabels = []


for i in range(len(train_random)):
    TrainData.append(train_random[i][0])
    TrainLabels.append(train_random[i][1])
    

for i in range(len(test_random)):
    TestData.append(test_random[i][0])
    TestLabels.append(test_random[i][1])

Dictionary

In [15]:
def dictionary(input_data):
    voc = {}
    tot_words = 0
    for i in range(len(input_data)):
        for w in input_data[i][0].split(" "):
            tot_words += 1
            if(w in voc):
                voc[w] += 1
            else:
                voc[w] = 1
    
    return voc, tot_words

In [16]:
voc, tot_words = dictionary(train)

Prior Calc

In [17]:
def prior_calc(trainClass, data):
    L = math.log(len(trainClass)/len(data))
    
    return L

In [18]:
L0 = prior_calc(train0, train)
L1 = prior_calc(train1, train)

Prediction

In [19]:
def predict(mL0, mL1, token0, token1, test):
    y = []
    acc = 0
    
    for i in range(len(test)):
        pred0 = mL0
        pred1 = mL1
    
        for w in test[i][0].split(" "):
            w = w.lower()
            if token0.__contains__(w):
                pred0 += token0[w]
            if token1.__contains__(w):    
                pred1 += token1[w]
    
        if pred0 > pred1:
            y.append(0)
        else: 
            y.append(1)
        
        if y[i] == test[i][1]:
            acc += 1
    
    total_acc = acc/len(test)
    
    return total_acc

mAP calculation

In [20]:
def Map(voc_data, voc_class, tot_words, k):
    Acc_tokens = {}
    voc_len = len(voc_data)
    
    for w in voc_data:
        if voc_class.__contains__(w):
            Acc_tokens[w] = math.log((voc_class[w] + k)/(tot_words + (voc_len*k)))
        else:
            if k != 0:
                Acc_tokens[w] = math.log((k)/(tot_words + (voc_len*k)))
    return Acc_tokens

In [21]:
L_token0 = Map(voc, word_0, tot_word0, 0)
L_token1 = Map(voc, word_1, tot_word1, 0)

In [22]:
Map_token0 = Map(voc, word_0, tot_word0, 1)
Map_token0 = Map(voc, word_1, tot_word1, 1)

Cross Validation

In [ ]:
def kfold_cross_validation(t0,t1,a,m,k):
    
    new_train_0 = random.sample(t0,len(t0))
    new_train_1 = random.sample(t1,len(t1))
    
    fold = int(len(new_train_0)/a)
    
    MAP_accuracy = np.zeros((10, 10))
    
    MAP_sm_accuracy = []
    
    sd_L = []
    sd_map = []

    cv_test0 = []
    cv_test1 = []
    cv_train0 = []
    cv_train1 = []

    for i in range(a):
        if i == 0:
            cv_test0 = new_train_0[(i*fold):((i+1)*fold)]
            cv_train0 = new_train_0[(i+1)*fold:]

            cv_test1 = new_train_1[i*fold:(i+1)*fold]
            cv_train1 = new_train_1[(i+1)*fold:]
        if i == (a-1):
            cv_test0 = new_train_0[i*fold:(i+1)*fold]
            cv_train0 = new_train_0[:(i)*fold]

            cv_test1 = new_train_1[i*fold:(i+1)*fold]
            cv_train1 = new_train_1[:(i)*fold]
        else:
            cv_test0 = new_train_0[i*fold:(i+1)*fold]
            cv_train0 = new_train_0[:(i)*fold] + new_train_0[(i+1)*fold:]

            cv_test1 = new_train_1[i*fold:(i+1)*fold]
            cv_train1 = new_train_1[:(i)*fold] + new_train_1[(i+1)*fold:]
            
            
        # Now we have cv_test_0, cv_train_0, cv_test_1, cv_train_1
        if q == 1:
            for l in range(10):

                new_train_1 = cv_train_1[:int(((l+1)*len(cv_train_1)/10))]
                new_train_0 = cv_train_0[:int(((l+1)*len(cv_train_0)/10))]

                vocab,total_words = create_dict(new_train_1 + new_train_0)

                vocab_1,total_words_1 = create_dict(new_train_1)
                vocab_0,total_words_0 = create_dict(new_train_0)

                maxL_0 = generate_prior(new_train_0,new_train_1+new_train_0)
                maxL_1 = generate_prior(new_train_1,new_train_1+new_train_0)

                MAP_tokens_0 = calculate_MAP(vocab,vocab_0,total_words_0,m)
                MAP_tokens_1 = calculate_MAP(vocab,vocab_1,total_words_1,m)

                MAP_curve_accuracy[i][l] = prediction(maxL_0,maxL_1,MAP_tokens_0,MAP_tokens_1,cv_test_1+cv_test_0)
                
            
        if q == 2:
            new_train_1 = cv_train_1
            new_train_0 = cv_train_0

            vocab,total_words = create_dict(new_train_1 + new_train_0)

            vocab_1,total_words_1 = create_dict(new_train_1)
            vocab_0,total_words_0 = create_dict(new_train_0)

            maxL_0 = generate_prior(new_train_0,new_train_1+new_train_0)
            maxL_1 = generate_prior(new_train_1,new_train_1+new_train_0)

            MAP_tokens_0 = calculate_MAP(vocab,vocab_0,total_words_0,m)
            MAP_tokens_1 = calculate_MAP(vocab,vocab_1,total_words_1,m)

            MAP_smooth_accuracy.append(prediction(maxL_0,maxL_1,MAP_tokens_0,MAP_tokens_1,cv_test_1+cv_test_0))
    
    if q == 2:
        mean = np.mean(MAP_smooth_accuracy,axis=0)
        sd = np.std(MAP_smooth_accuracy,axis=0)
        return mean,sd
    
    if q ==1:
        mean = np.mean(MAP_curve_accuracy, axis=0)
        sd = np.std(MAP_curve_accuracy, axis=0)
        return mean,sd